## Applying six basic ML algorithms to the extracted features 

In [1]:
import os
import numpy as np
import pandas as pd
import mne
import mne_features
import matplotlib.pyplot as plt
%matplotlib qt

# Importing CSV data and appedning data frames

In [2]:
root = "../Dataset/"

In [3]:
bckg_data = pd.read_csv('../Dataset/bckg_features_data0.csv') # CSV of bckg 0 to 75

D:\Tools\Anaconda\envs\tensorflow-gpu\lib\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,519,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,548,549,550,551,552,553,554,555,556,557,558,559,560,561,562,563,564,565,566,567,568,569,570,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,598,599,600,601,602,603,604,605,606,607,608) have mixed types.Specify dtype option on 

In [4]:
bckg_data.shape

(2230, 631)

In [5]:
bckg_data2 = pd.read_csv('../Dataset/bckg_features_data.csv') # CSV of bckg 75 to 150

In [6]:
bckg_data2.shape

(9732, 631)

In [7]:
bckg_data = bckg_data.append(bckg_data2.iloc[1: , :], ignore_index=True) # Appeding bckg data

In [8]:
#bckg_data.sample(frac=1).reset_index(drop=True)

In [9]:
gnsz_data = pd.read_csv('../Dataset/gnsz_features_data.csv') # all gnsz CSV data

In [10]:
#gnsz_data.sample(frac=1).reset_index(drop=True)

In [11]:
print(bckg_data.iloc[1: , :].shape, gnsz_data.iloc[1: , :].shape)

(11960, 631) (15981, 631)


In [12]:
df = pd.DataFrame()

In [13]:
df = df.append(gnsz_data.iloc[1: , :], ignore_index=True) # Appending gnsz data

In [14]:
df = df.append(bckg_data.iloc[1: , :], ignore_index=True) # Appending bckg data

In [15]:
df = df.drop(df.columns[0:4*21], axis=1)

In [16]:
#df.head()

In [17]:
df = df.apply(pd.to_numeric)

In [18]:
df.isnull().values.any()

False

In [19]:
df.shape

(27941, 547)

In [20]:
#(df-df.mean())/df.std()

# Testing ML models

## Data Matrix

In [21]:
#df = df.sample(frac = 1)

In [22]:
X = df.drop(['output'], axis=1)
Y = df['output']

In [23]:
Y.shape

(27941,)

In [24]:
X = (X-X.mean())/X.std()

In [25]:
#from sklearn.utils import shuffle
#X, Y = shuffle(X, Y, random_state=0)

In [33]:
from sklearn.model_selection import KFold 

k = 5
kf = KFold(n_splits=k, random_state=None, shuffle=True)

from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import accuracy_score

In [27]:
#from sklearn.model_selection import train_test_split
#X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=0)

## Logistic Regression

In [126]:
acc_score = []

model = LogisticRegression(penalty='l2', C=0.01, max_iter=5000)
for train_index , test_index in kf.split(X,Y):
    X_train , X_test = X.iloc[train_index,:],X.iloc[test_index,:]
    y_train , y_test = Y[train_index] , Y[test_index]
     
    model.fit(X_train,y_train)
    pred_values = model.predict(X_test)
     
    acc = accuracy_score(pred_values , y_test)
    acc_score.append(acc)
     
avg_acc_score = sum(acc_score)/k
 
print('accuracy of each fold - {}'.format(acc_score))
print('Avg accuracy : {}'.format(avg_acc_score))

accuracy of each fold - [0.9284308463052424, 0.9205440229062276, 0.9232283464566929, 0.9178596993557624, 0.9243020758768791]
Avg accuracy : 0.9228729981801609


In [127]:
print('confusion_matrix\n',metrics.confusion_matrix(y_test,pred_values))

confusion_matrix
 [[2197  213]
 [ 210 2968]]


In [128]:
print('classification_report\n',metrics.classification_report(y_test,pred_values))

classification_report
               precision    recall  f1-score   support

         0.0       0.91      0.91      0.91      2410
         1.0       0.93      0.93      0.93      3178

    accuracy                           0.92      5588
   macro avg       0.92      0.92      0.92      5588
weighted avg       0.92      0.92      0.92      5588



In [129]:
import seaborn as sns
sns.heatmap(metrics.confusion_matrix(y_test,pred_values), annot=True, 
            fmt='', cmap='Blues')

<AxesSubplot:>

## K-Nearest Neighbours

In [59]:
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier(n_neighbors=128) #16 or 1 or null(+)

acc_score = []

for train_index , test_index in kf.split(X,Y):
    X_train , X_test = X.iloc[train_index,:],X.iloc[test_index,:]
    y_train , y_test = Y[train_index] , Y[test_index]
     
    model.fit(X_train,y_train)
    pred_values = model.predict(X_test)
     
    acc = accuracy_score(pred_values , y_test)
    acc_score.append(acc)
     
avg_acc_score = sum(acc_score)/k
 
print('accuracy of each fold - {}'.format(acc_score))
print('Avg accuracy : {}'.format(avg_acc_score))

accuracy of each fold - [0.9092860976918948, 0.9144595561918396, 0.9114173228346457, 0.9101646385110952, 0.9128489620615605]
Avg accuracy : 0.9116353154582072


In [60]:
print('confusion_matrix\n',metrics.confusion_matrix(y_test,pred_values))

confusion_matrix
 [[2088  262]
 [ 225 3013]]


In [61]:
print('classification_report\n',metrics.classification_report(y_test,pred_values))

classification_report
               precision    recall  f1-score   support

         0.0       0.90      0.89      0.90      2350
         1.0       0.92      0.93      0.93      3238

    accuracy                           0.91      5588
   macro avg       0.91      0.91      0.91      5588
weighted avg       0.91      0.91      0.91      5588



In [62]:
import seaborn as sns
sns.heatmap(metrics.confusion_matrix(y_test,pred_values), annot=True, 
            fmt='', cmap='Blues')

<AxesSubplot:>

## Decision Tree Classifier

In [69]:
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier()

acc_score = []

for train_index , test_index in kf.split(X,Y):
    X_train , X_test = X.iloc[train_index,:],X.iloc[test_index,:]
    y_train , y_test = Y[train_index] , Y[test_index]
     
    model.fit(X_train,y_train)
    pred_values = model.predict(X_test)
     
    acc = accuracy_score(pred_values , y_test)
    acc_score.append(acc)
     
avg_acc_score = sum(acc_score)/k
 
print('accuracy of each fold - {}'.format(acc_score))
print('Avg accuracy : {}'.format(avg_acc_score))

accuracy of each fold - [0.9171587045983182, 0.9185755189692197, 0.9130279169649248, 0.920722977809592, 0.9119541875447388]
Avg accuracy : 0.9162878611773587


In [70]:
print('confusion_matrix\n',metrics.confusion_matrix(y_test,pred_values))

confusion_matrix
 [[2151  238]
 [ 254 2945]]


In [71]:
print('classification_report\n',metrics.classification_report(y_test,pred_values))

classification_report
               precision    recall  f1-score   support

         0.0       0.89      0.90      0.90      2389
         1.0       0.93      0.92      0.92      3199

    accuracy                           0.91      5588
   macro avg       0.91      0.91      0.91      5588
weighted avg       0.91      0.91      0.91      5588



In [72]:
import seaborn as sns
sns.heatmap(metrics.confusion_matrix(y_test,pred_values), annot=True, 
            fmt='', cmap='Blues')

<AxesSubplot:>

## Random Forest

In [79]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(max_depth=5)

acc_score = []

for train_index , test_index in kf.split(X,Y):
    X_train , X_test = X.iloc[train_index,:],X.iloc[test_index,:]
    y_train , y_test = Y[train_index] , Y[test_index]
     
    model.fit(X_train,y_train)
    pred_values = model.predict(X_test)
     
    acc = accuracy_score(pred_values , y_test)
    acc_score.append(acc)
     
avg_acc_score = sum(acc_score)/k
 
print('accuracy of each fold - {}'.format(acc_score))
print('Avg accuracy : {}'.format(avg_acc_score))

accuracy of each fold - [0.9040973340490249, 0.9073013600572656, 0.9028274874731568, 0.9083750894774517, 0.9001431639226914]
Avg accuracy : 0.9045488869959181


In [80]:
print('confusion_matrix\n',metrics.confusion_matrix(y_test,pred_values))

confusion_matrix
 [[1899  441]
 [ 117 3131]]


In [81]:
print('classification_report\n',metrics.classification_report(y_test,pred_values))

classification_report
               precision    recall  f1-score   support

         0.0       0.94      0.81      0.87      2340
         1.0       0.88      0.96      0.92      3248

    accuracy                           0.90      5588
   macro avg       0.91      0.89      0.90      5588
weighted avg       0.90      0.90      0.90      5588



In [82]:
import seaborn as sns
sns.heatmap(metrics.confusion_matrix(y_test,pred_values), annot=True, 
            fmt='', cmap='Blues')

<AxesSubplot:>

## Support Vector Classifier

In [121]:
from sklearn.svm import SVC
model = SVC(C=0.25)

acc_score = []

for train_index , test_index in kf.split(X,Y):
    X_train , X_test = X.iloc[train_index,:],X.iloc[test_index,:]
    y_train , y_test = Y[train_index] , Y[test_index]
     
    model.fit(X_train,y_train)
    pred_values = model.predict(X_test)
     
    acc = accuracy_score(pred_values , y_test)
    acc_score.append(acc)
     
avg_acc_score = sum(acc_score)/k
 
print('accuracy of each fold - {}'.format(acc_score))
print('Avg accuracy : {}'.format(avg_acc_score))

accuracy of each fold - [0.9348720701377706, 0.9371868289191124, 0.9311023622047244, 0.9332498210450967, 0.9357551896921976]
Avg accuracy : 0.9344332543997804


In [116]:
print('confusion_matrix\n',metrics.confusion_matrix(y_test,pred_values))

confusion_matrix
 [[2159  204]
 [ 205 3020]]


In [115]:
print('classification_report\n',metrics.classification_report(y_test,pred_values))

classification_report
               precision    recall  f1-score   support

         0.0       0.91      0.91      0.91      2363
         1.0       0.94      0.94      0.94      3225

    accuracy                           0.93      5588
   macro avg       0.93      0.93      0.93      5588
weighted avg       0.93      0.93      0.93      5588



In [124]:
import seaborn as sns
sns.heatmap(metrics.confusion_matrix(y_test,pred_values), annot=True, 
            fmt='', cmap='Blues')

<AxesSubplot:>

## Linear Discriminant Analysis

In [106]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

model = LinearDiscriminantAnalysis(solver='eigen', shrinkage=0.5)

acc_score = []

for train_index , test_index in kf.split(X,Y):
    X_train , X_test = X.iloc[train_index,:],X.iloc[test_index,:]
    y_train , y_test = Y[train_index] , Y[test_index]
     
    model.fit(X_train,y_train)
    pred_values = model.predict(X_test)
     
    acc = accuracy_score(pred_values , y_test)
    acc_score.append(acc)
     
avg_acc_score = sum(acc_score)/k
 
print('accuracy of each fold - {}'.format(acc_score))
print('Avg accuracy : {}'.format(avg_acc_score))

accuracy of each fold - [0.8969404186795491, 0.8960272011453114, 0.8960272011453114, 0.8863636363636364, 0.8908375089477452]
Avg accuracy : 0.8932391932563106


In [111]:
print('confusion_matrix\n',metrics.confusion_matrix(y_test,pred_values))

confusion_matrix
 [[2118  295]
 [ 315 2860]]


In [112]:
print('classification_report\n',metrics.classification_report(y_test,pred_values))

classification_report
               precision    recall  f1-score   support

         0.0       0.87      0.88      0.87      2413
         1.0       0.91      0.90      0.90      3175

    accuracy                           0.89      5588
   macro avg       0.89      0.89      0.89      5588
weighted avg       0.89      0.89      0.89      5588



In [113]:
import seaborn as sns
sns.heatmap(metrics.confusion_matrix(y_test,pred_values), annot=True, 
            fmt='', cmap='Blues')

<AxesSubplot:>